In [ ]:

from core.core_log import setup_logging, get_logger
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime, timedelta
from core.database import SensorDataHandler
from core.plotting import draw_plots
from core.util import create_timelapse
from core.sensors.camera import RpiCamController, DIMENSION_FULL_HD, DIMENSION_CAMERA_DEFAULT
from core.sensors.dht import get_sensor_data
from core.core_configuration import load_config, database_config, hometemp_config

setup_logging()
load_config()
log = get_logger(__name__)

In [ ]:
#live_sensor_data = get_sensor_data(int(hometemp_config()["sensor_pin"]), True)
#live_sensor_data

In [ ]:
rpi_cam = RpiCamController()
#name = f'pictures/{datetime.now().strftime("%Y-%m-%d-%H:%M:%S")}'
name = "current"
#rpi_cam.capture_image(file_path=name)
#create_timelapse('./pictures', 'timelapse.mp4')

def _rotate_image(image_path: str, rotation: int) -> bool:
    if rotation != 0:
        image = Image.open(image_path)
        # 'expand' to resize for the whole image
        rotated_image = image.rotate(rotation, expand=True)
        rotated_image.save(image_path)
    return True

#rot = ["2024-10-08-21:50:40.png"]#, "2024-10-08-23:00:00.png", "2024-10-09-04:00:00.png", "2024-10-09-23:00:00.png", "2024-10-10-04:00:00.png", "2024-10-10-23:00:00.png", "2024-10-11-04:00:00.png", "2024-10-11-23:00:00.png"]
#map(lambda x: _rotate_image(f"pictures/{x}", 90), rot)

In [ ]:
auth = database_config()
handler = SensorDataHandler(auth['db_port'], auth['db_host'], auth['db_user'], auth['db_pw'], 'sensor_data')
handler.init_db_connection(check_table=False)
df = handler.read_data_into_dataframe()
df['timestamp'] = df['timestamp'].map(lambda x : datetime.strptime(str(x).replace("+00:00", "").strip(), '%Y-%m-%d %H:%M:%S'))
df = df.sort_values(by='timestamp')
df = df.reset_index()

# started automation on 3 Sep 2024 19:15 and ended 29 Nov 2024 16:10
df = df[(df['timestamp'] >= datetime(year=2024, month=9, day=3, hour=19)) & (df['timestamp'] <= datetime(year=2024, month=11, day=29, hour=16))]
df

In [ ]:
# last two hours
#df[["timestamp", "room_temp"]].tail(12)

In [ ]:
df[["humidity", "room_temp", "cpu_temp"]].describe()

In [ ]:
corr_matrix = df[["humidity", "room_temp", "cpu_temp"]].corr()
#corr_matrix = df[["humidity", "room_temp"]].corr()
corr_matrix

In [ ]:
draw_plots(df, with_save=False)

In [ ]:
def foo():
    plt.figure(figsize=(25, 10))
    plt.subplot(2, 1, 1)

    sns.lineplot(label="CPU", x="timestamp", y="cpu_temp", data=df)
    plt.title("Temperature Over Time")
    plt.xlabel("Time")
    plt.ylabel("Temp (°C)")
    plt.legend()
    plt.xticks(rotation=45)

    plt.subplot(2, 1, 2)
    sns.lineplot(label="CPU", x="timestamp", y="cpu_temp",marker='o',markersize=6, data=df[df["timestamp"] >= datetime.now() - timedelta(hours=25)])
    plt.title("Temperature Last 24h")
    plt.xlabel("Time")
    plt.ylabel("Temp (°C)")
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

#foo()
df